In [1]:
# TO connect with anvil server and import media and tables from anvil 

import anvil.server
anvil.server.connect("MPDLFTJHTQIFU5EXLAHYDVEW-Q4MVKUCLVQXKOMRJ")
import anvil.media
from anvil.tables import app_tables

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [2]:
# importing all necessary libraries for face recognition

import face_recognition
import cv2
import numpy as np

In [3]:
# creating global arrays for storing criminal id, known names and encoding of those known named faces from database stored
# on anvil server

criminal_id=[]
known_name=[]
known_face_encoding=[]

In [4]:
# loading and storing criminal id, criminal names and their face encodings from database stored on anvil server 

for row in app_tables.criminal_record.search():     #accessing rows via search()
    criminal_id.append(row['criminal_id'])
    known_name.append(row['name'])
    with anvil.media.TempFile(row['image']) as filename:
        image=face_recognition.load_image_file(filename)
        image_encoding=face_recognition.face_encodings(image)[0]
        known_face_encoding.append(image_encoding)
    

In [5]:
# function called when new criminal data added in database to append new criminal'd id,name and face encoding in the above
# enlisted global arrays

@anvil.server.callable
def add_in(name,c_id,file):
    
    criminal_id.append(c_id)
    known_name.append(name)
    
    with anvil.media.TempFile(file) as filename:
        file=face_recognition.load_image_file(filename)
        image_encoding=face_recognition.face_encodings(file)[0]
        known_face_encoding.append(image_encoding)
        
    

In [6]:
# To check whether the uploaded image of the person on the website matches with the faces of the criminals stored in 
# database and return its name and criminal id if found else return not found if face doesnt match with any criminal and 
# just in case if no face gets detected on the uploaded images it will return No face detected so that user can load another
# image of the person to check.

@anvil.server.callable
def search(file):
    name="Unknown, Not in database."
    c_id="Null"
    
    face_locations = []
    face_encodings = []
    
    face_names = []
    face_id=[]
    
    with anvil.media.TempFile(file) as filename:
        file=face_recognition.load_image_file(filename)
        
    small_file = cv2.resize(file, (0, 0), fx=0.25, fy=0.25)
    try:     # if face gets detected
        face_encodings=face_recognition.face_encodings(small_file)[0]   

    except: # face doesnt gets detected
        name = "No Face detected in the Uploaded Image. Try Uploading Another Image !"
        return name,c_id  # returns from here
    
  #continues if face gets detected  
    matches = face_recognition.compare_faces(known_face_encoding, face_encodings)

    face_distances = face_recognition.face_distance(known_face_encoding, face_encodings)
    best_match_index = np.argmin(face_distances)
    if matches[best_match_index]:
        name = known_name[best_match_index]
        c_id=criminal_id[best_match_index]
            
    return name,c_id
                
            
    
                
            
    

In [7]:
# if face gets detected on the uploaded image then returns its encoding else returns fails. Called while live tracking of 
# person who may or may not be a criminal 

@anvil.server.callable
def track_encoding(file):
   
    with anvil.media.TempFile(file) as filename:
        file=face_recognition.load_image_file(filename)
        
    small_file = cv2.resize(file, (0, 0), fx=0.25, fy=0.25)
    try:
        track_face_encoding=face_recognition.face_encodings(small_file)[0]
    except:
        track_face_encoding="fail"
    return track_face_encoding

In [8]:
# does Live streaming and displays detected if uploaded image matches the live streamed persons face else displays unknown 
# here person may or may not be criminal. This function is used to track down specific person whose image is uploaded

@anvil.server.callable
def track_criminal(track_face_encoding,stop):
    face_locations = []
    face_encodings = []
    face_names = []
    
    # Get a reference to webcam #0 (the default one)
    video_capture = cv2.VideoCapture(0)
    while True:
        
        # Hit 'q' on the keyboard to quit and to release all connections !
        if cv2.waitKey(1) & 0xFF == ord('q'):
            video_capture.release()
            cv2.destroyAllWindows()
            break
        
        # Grab a single frame of video
        ret, frame = video_capture.read()
        
        if ret: # if frame was returned 
            
            # Resize frame of video to 1/4 size for faster face recognition processing
            small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
            
            # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
            rgb_small_frame = small_frame[:, :, ::-1]
            
             # Find all the faces and face encodings in the current frame 
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            for face_encoding in face_encodings:   # iterating over all detected face's encoding
                
                # See if the face is a match for the known face(s)
                matches = face_recognition.compare_faces([track_face_encoding], face_encoding)
                name = "Unknown"
                
                # use the known face with the smallest distance to the new face
                face_distances = face_recognition.face_distance([track_face_encoding], face_encoding)
                best_match_index = np.argmin(face_distances)   # finding index of the minimum distance
                if matches[best_match_index]:
                    name = "Detected"
                face_names.append(name)
                
                # Display the results
            for (top, right, bottom, left), name in zip(face_locations, face_names):
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4

                # Draw a box around the face
                cv2.rectangle(frame, (left, top), (right, bottom), (255, 153, 153), 2)

                # Draw a label with a name below the face
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (255, 153, 153), cv2.FILLED)
                font = cv2.FONT_HERSHEY_COMPLEX
                cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (53, 255, 153), 1)

             # Display the resulting image
            cv2.imshow('Video', frame)
        else: # if frame doesnt gets returned
            break
                
    

In [9]:
# displays name and criminal id if the detected faces via live streaming matches the faces in the criminal databse. This 
# detects every face in the live streamed captured frames and matches then with the faces in the database. 

@anvil.server.callable
def survillence():
    
    face_locations = []
    face_encodings = []
    
    
    face_names = []
    face_id=[]
    
    video_capture = cv2.VideoCapture(0)
    
    
    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            video_capture.release()
            cv2.destroyAllWindows()
            break
        
       
        ret, frame = video_capture.read()
        
        if ret:
           
            small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
           
            rgb_small_frame = small_frame[:, :, ::-1]

            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(known_face_encoding, face_encoding)
                name = "Unknown"
                c_id="Null"

                face_distances = face_recognition.face_distance(known_face_encoding, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name =  known_name[best_match_index]
                    c_id = criminal_id[best_match_index]
                face_names.append(name)
                face_id.append(c_id)
                
                
            for (top, right, bottom, left),name,c_id in zip(face_locations,face_names,face_id):
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4

                cv2.rectangle(frame, (left, top), (right, bottom), (255,51,51), 2)

                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (255,51,51), cv2.FILLED)
                font = cv2.FONT_HERSHEY_PLAIN
                text="Name ="+name+" C_ID = "+str(c_id)
                
                cv2.putText(frame,text, (left + 6, bottom - 6), font, 1.0, (0,255,0), 1)

            cv2.imshow('Survillence', frame)
        else:
            break